Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
# Import any packages you need here
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

#Creating the literal class
class Literal:
    
    #Initializing with a name and a sign that is true by default
    def __init__(self, name, sign=True):
        
        self.name = name
        self.sign = sign
        
    #This method enables us to change the truth value of the sign attribute
    #when a minus is put before the literal
    def __neg__(self):
        return Literal(self.name, not self.sign)
    
    #Representation of the literals
    def __repr__(self):
        #If it's negative we print it with a minus before it
        if self.sign == False:
            return '-' + self.name
        #If it is positive we just print the name
        else:
            return self.name
        
    #Defining equality between literals
    def __eq__(self, other):
        return(self.name == other.name)
    
    #Defining greatness relations. I used this for alphabetically sorting the array
    def __gt__(self, other):
        return(self.name > other.name)
    
    #Creating a hash. I had errors for not haaving a hash while writing the code, so I added this.
    def __hash__(self):
        return(hash(self.name))

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [2]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below
# YOUR CODE HERE (OPTIONAL)

def is_true_model(clause, model):
    '''
    Takes in a clause and a model.
    Outputs whether the clause is true for the given model
    '''
    #Looping over the literals
    for literal in clause:
        
        #If the literal exists in the model
        if literal in model:
            
            #If the sign and the truth value assigned match it means the truth val is True.
            #If there is one True literal the whole clause becomes True.
            if literal.sign == model[literal]:
                return True
    #If no True literals are found, we cannot confirm this
    return False


def check_all_true(KB, model):
    '''
    Takes in a KB and a model
    Checks if all the clauses are true at a given time.
    '''
    #We assume it is true
    value=True
    
    #we loop through all clauses
    for clause in KB:
        
        #If we find one clause that cannot be confirmed to be true we return false and break
        if not is_true_model(clause, model):
            value=False
            break
    return value


def get_symbols(KB):
    '''
    Function to get a list of symbols from a knowledge base
    '''
    
    #Initializng an empty list
    symbols=[]
    
    #For each clause
    for cnf in KB:
        
        #For each literal
        for literal in cnf:
            
            #We take the literal without a sign
            if literal.sign == True:
                symbols.append(literal)
            else:
                symbols.append(-literal)
    
    #We use set to get every literal ones. We turn them into a list. We sort them alphabetically
    return sorted(list(set(symbols)))

def DPLL_Satisfiable(KB, heuristic_level=0):
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    #Creating a model dictionary
    model = {}
    
    #Runs the DPLL function with symbols extracted
    return DPLL(KB, get_symbols(KB), model, [], heuristic_level)


def DPLL(KB, symbols, model, chosen_symbols, heuristic_level=0):
    '''
    KB - knowledge base, symbols - current list of symbols,
    model - the current model, chosen_symbols - symbols that are assigned a truth value for the solution,
    heuristic_level - which heuristic to use
    
    returns - Satisfiability truth val, the model, chosen_symbols
    '''
    
    #First making sure that the KB is in its simplest form for the given model
    #This eliminates the clauses that are true for the current model
    KB = [clause for clause in KB if is_true_model(clause, model) != True]
    
    #We pass the chosen symbols inside
    chosen_symbols = chosen_symbols
    
    #If there are clauses in the KB, if not it means we solved the problem
    if KB:
        #The degree heuristic is used if the heuristic_level is 1    
        if heuristic_level == 1:
            symbols = degree_heuristic(KB, symbols)

        #Checks whether all the clauses are true. If so returns true
        if check_all_true(KB, model):
            return (True, model, chosen_symbols)

    #If we dont have any clauses in the KB it also returns true since everything is satisfied
    elif not KB:
        return (True, model, chosen_symbols)
    
    #If we have unassigned symbols
    if symbols:
        
        #I copy the symbols and the KB to call in the recursions
        new_KB = KB.copy()
        new_symbols = symbols.copy()
        

        #We pop from the list. (If a heuristic is used the list is already ordered accordingly)
        P = new_symbols.pop(0)
        
        
        #Making a copy of the model
        true_model = model.copy()
        
        #Setting the value of P to true
        true_model[P] = True
        
        #Calling the function recursively with the newly assigned literal
        res1 = DPLL(KB=new_KB, symbols=new_symbols, model=true_model, heuristic_level=heuristic_level, chosen_symbols=chosen_symbols)
        
        #If this returns true, that means we have found a solution at a level and we want to pass this to
        #lower levels of the recursion. Therefore, we return the same output
        if res1[0] == True:
            chosen_symbols.insert(0, P)
            return res1[0], res1[1], chosen_symbols

        chosen_symbols = res1[2]
        
        #Using another copy for the false model
        false_model = model.copy()
        
        #Setting the value of P to false
        false_model[P] = False
        
        #Calling the function recursively with the newly assigned literal
        res2 = DPLL(KB=new_KB, symbols=new_symbols, model=false_model, heuristic_level=heuristic_level, chosen_symbols=chosen_symbols)
        
        #If this returns true, that means we have found a solution at a level and we want to pass this to
        #lower levels of the recursion. Therefore, we return the same output
        if res2[0] == True:
            chosen_symbols.insert(0, P)
            return res2[0], res2[1], chosen_symbols
     
    #If the code reaches here, it means there are no solutions.
    return (False, model, chosen_symbols)
    
    
    
    

## Degree Heuristic

- I did my own implementation of the degree heuristic below. My output of the visited states doesn't match with the ones provided in the test case below, but I still wanted to add my code here since I think this was a worthy try. My code does what I intuitively think it should do. I thought about this for a while. Even though I'm not entirely clear, I feel like my code is exploring an asymmetrical binary tree, whereas the output from the code below looks like it explores the same nodes previously explored when a parent was true. I'm also not sure if this is the root cause of the problem or if mine also works or maybe potentially it works better. So I decided to include it here and leave the reast to your judgment.

- The reasoning my degree heuristic uses is that, whenever the DPLL algorithm is called it simplifies the clauses that would have a truth value of 1 with the given model. Then counts the occurences of literals in the KB, and sorts the symbols list based on this count.

In [3]:
#Degree heuristic function

def degree_heuristic(KB, symbol_list):
    '''
    Takes is a KB, a list of symbols and a list of symbols chosen so far
    Outputs a sorted symbol list based on the number of occurances
    '''
    
    #creating an empty dictionary for counting
    a = {}
    
    #Initializing counts for every unassigned symbol
    for sym in symbol_list:
        a[sym] = 0
        
    #Looping over each clause in the KB
    for clause in KB:
        #Looping over every literal within clauses
        for literal in clause:
            #if the literal is in a
            if literal in a:
                #we increase its count
                a[literal] += 1
    
    #We create a sorted list using these counts
    new_symbol_list = sorted(symbol_list, key=a.get, reverse=True)
    
    #Returns the new symbol list
    return new_symbol_list

In [4]:
#Testing the degree heuristic function to see if it returns with the correct order
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

#Symbol List for testing
al= [A, B, C, D, E, F]

#Using the KB from below
KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F} ,{-B, C}]

degree_heuristic(KB, al)

[B, E, A, C, F, D]

This is what it returns when it is run on the example from Q3

In [5]:
#Testing on the example test case from Q3

A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F} ,{-B, C}]

DPLL_Satisfiable(KB,1)

(True, {B: False, E: False, A: False}, [B, E, A])

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [6]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

# YOUR CODE HERE
#I used biconditional elimination and material implication to get the clauses be disjunctions. There was a conjunction of
#Two disjuctions. I added it to the KB as two disjunctions.

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F} ,{-B, C}]

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [7]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [8]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [9]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [10]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{A: False, B: False, C: True, D: True, E: False}


<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [11]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], ['B', 'A', 'C', 'F', 'E', 'A']])

symbol_trace

{B: False, E: False, A: False}


AssertionError: 

In [12]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'])

{A: False, B: False, C: True, D: True, E: False}


AssertionError: 

In [13]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])

{A: False, B: False, C: True, D: True, E: False}
[A, B, C, D, E]


AssertionError: 